In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor,IsolationForest,ExtraTreesRegressor,GradientBoostingRegressor

from sklearn import preprocessing
import scoring as score # para hacer los reportes de puntajes
from time import time

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/properati_final_y_descripcion.csv')
zonas = pd.read_csv('datos/todos_los_barrios.csv')

tipo_puntaje="r2"
iteraciones_de_searchs=20
cv=5

<IPython.core.display.Javascript object>

# Preprocesamiento

In [5]:
#paso features con strings a numericos
le_zona = preprocessing.LabelEncoder()
properati['state_name'] =le_zona.fit_transform(properati['state_name'])

le_barrio = preprocessing.LabelEncoder()
le_barrio.fit(zonas.values)
properati['place_name'] = le_barrio.transform(properati['place_name'])

le_tipo = preprocessing.LabelEncoder()
properati['property_type'] = le_tipo.fit_transform(properati['property_type'])

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
properati=properati.loc[properati.price_aprox_usd<1e6]
properati=properati.loc[properati.surface_covered_in_m2<7000]
print len(properati)
properati=properati.groupby("place_name").filter(lambda x: len(x)>5)
print len(properati)

133183
132784


In [7]:
#preparo set de datos
#X = zip(properati['surface_total_in_m2'],\
       # properati['surface_covered_in_m2'],properati['place_name'],properati["cochera"],properati["lavadero"],properati["patio"],properati["terraza"],properati["hall"]\
        #,properati["lavadero"])
        #properati['surface_covered_in_m2'],properati['place_name'])
originales=["surface_total_in_m2","surface_covered_in_m2","place_name","property_type"]
features_nuevas=["cochera","comedor","living","lavadero","terraza","hall","placard","balcon","parrilla","aire","patio","pileta"\
                ,"mesada","parque","quincho","escalera","monoambiente","caldera","solarium","duplex"]
columnas=originales+features_nuevas
X= properati.loc[:,columnas]    
y = properati["price_aprox_usd"]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132784 entries, 0 to 136351
Data columns (total 24 columns):
surface_total_in_m2      132784 non-null float64
surface_covered_in_m2    132784 non-null float64
place_name               132784 non-null int64
property_type            132784 non-null int64
cochera                  132784 non-null bool
comedor                  132784 non-null bool
living                   132784 non-null bool
lavadero                 132784 non-null bool
terraza                  132784 non-null bool
hall                     132784 non-null bool
placard                  132784 non-null bool
balcon                   132784 non-null bool
parrilla                 132784 non-null bool
aire                     132784 non-null bool
patio                    132784 non-null bool
pileta                   132784 non-null bool
mesada                   132784 non-null bool
parque                   132784 non-null bool
quincho                  132784 non-null bool
escaler

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

# XGBoost

In [40]:
%%notify
from xgboost import XGBRegressor
xgb= XGBRegressor(nthread=-1)

learning_rate=np.arange(0.3,0.9,0.2)
gamma = np.arange(0.1,0.9,0.1)
subsample = [1]

param_grid = {"learning_rate": learning_rate,'min_child_weight':[1,3],'gamma':gamma,'subsample':subsample}

search = RandomizedSearchCV(xgb, param_distributions=param_grid ,cv=cv,n_iter=iteraciones_de_searchs,scoring=tipo_puntaje) 
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 192.53 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.816 (std: 0.001)
Promedio validation score: 0.802 (std: 0.003)
Promedio fit time: 1.504s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.69999999999999996, 'gamma': 0.30000000000000004, 'min_child_weight': 3}

Puesto: 1
Promedio training score: 0.816 (std: 0.001)
Promedio validation score: 0.802 (std: 0.003)
Promedio fit time: 1.448s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.69999999999999996, 'gamma': 0.5, 'min_child_weight': 3}

Puesto: 3
Promedio training score: 0.815 (std: 0.001)
Promedio validation score: 0.800 (std: 0.003)
Promedio fit time: 1.708s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.69999999999999996, 'gamma': 0.40000000000000002, 'min_child_weight': 1}

Puesto: 3
Promedio training score: 0.815 (std: 0.001)
Promedio validation score: 0.800 (std: 0.003)
Promedio fit time: 1.466s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.699

<IPython.core.display.Javascript object>

In [41]:
xgboost = search.best_estimator_

# RF

In [ ]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["auto",None]
n_estimators = np.arange(40,100,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=cv,n_iter=iteraciones_de_searchs,scoring=tipo_puntaje) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

## Grid Search

In [42]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =[None]
n_estimators = np.arange(50,70,5)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(rf, param_grid=param_grid ,cv=cv,scoring=tipo_puntaje) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 173.47 segundos para 4 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.824 (std: 0.003)
Promedio fit time: 7.651s
Hiper-parametros: {'max_features': None, 'n_estimators': 65, 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.823 (std: 0.003)
Promedio fit time: 7.340s
Hiper-parametros: {'max_features': None, 'n_estimators': 60, 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.823 (std: 0.003)
Promedio fit time: 6.510s
Hiper-parametros: {'max_features': None, 'n_estimators': 55, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.823 (std: 0.003)
Promedio fit time: 6.437s
Hiper-parametros: {'max_features': None, 'n_estimators': 50, 'criterion': 'mse'}



<IPython.core.display.Javascript object>

In [43]:
rf= search.best_estimator_

In [44]:
rf.feature_importances_

array([  7.83724881e-02,   6.39645009e-01,   9.56288937e-02,
         6.42328130e-02,   1.90092278e-02,   5.10117766e-03,
         5.42251320e-03,   6.37993662e-03,   6.54608450e-03,
         1.15905632e-02,   5.41784331e-03,   4.13065413e-03,
         7.11902960e-03,   6.41074070e-03,   8.25349991e-03,
         7.72396258e-03,   4.11086618e-03,   5.07406171e-03,
         5.18615437e-03,   3.81012338e-03,   4.81623621e-04,
         3.25339632e-03,   5.30332975e-03,   1.79600786e-03])

# ETR

In [45]:
%%notify

etr = ExtraTreesRegressor(n_jobs=-1)

n_estimators = np.arange(40,100,5)

param_grid = {"n_estimators":n_estimators}

search = GridSearchCV(etr, param_grid=param_grid ,cv=cv,scoring=tipo_puntaje) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 529.91 segundos para 12 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.799 (std: 0.001)
Promedio fit time: 8.393s
Hiper-parametros: {'n_estimators': 80}

Puesto: 2
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.799 (std: 0.001)
Promedio fit time: 8.857s
Hiper-parametros: {'n_estimators': 85}

Puesto: 3
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.799 (std: 0.001)
Promedio fit time: 9.202s
Hiper-parametros: {'n_estimators': 90}

Puesto: 4
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.799 (std: 0.001)
Promedio fit time: 9.649s
Hiper-parametros: {'n_estimators': 95}

Puesto: 5
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.798 (std: 0.001)
Promedio fit time: 7.580s
Hiper-parametros: {'n_estimators': 70}

Puesto: 6
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.798 (std: 0.0

<IPython.core.display.Javascript object>

In [46]:
etr= search.best_estimator_

In [47]:
etr.feature_importances_

array([ 0.1025775 ,  0.38431277,  0.06099997,  0.11601939,  0.02768446,
        0.01194931,  0.00912856,  0.03998314,  0.01297916,  0.03836263,
        0.01192696,  0.009114  ,  0.01586781,  0.01221404,  0.01458539,
        0.06537316,  0.01058924,  0.01012385,  0.00990585,  0.00781894,
        0.00793403,  0.00632625,  0.0085098 ,  0.0057138 ])

In [56]:
etr.scoreore(X_test,y_test)

0.81651248817297262

## Lineales

In [2]:
from sklearn.linear_model import Lasso,Ridge

In [9]:
lasso=Lasso()
lasso.fit(X_train,y_train)
lasso.score(X_test,y_test)

0.46498378198933171

In [10]:
ridge=Ridge()
ridge.fit(X_train,y_train)
ridge.score(X_test,y_test)

0.4649833332184411

# Ensamble

In [48]:
from mlxtend.regressor import StackingRegressor

ensamble = StackingRegressor(regressors=[rf,etr,lasso,ridge], 
                           meta_regressor=xgb)

In [49]:
%%notify
ensamble.fit(X_train,y_train)

StackingRegressor(meta_regressor=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
         regressors=[RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimato...stimators=80, n_jobs=-1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)],
         verbose=0)

<IPython.core.display.Javascript object>

In [50]:
ensamble.score(X_test,y_test)

0.81675992245466844

In [51]:
errores = ensamble.predict(X_test)-y_test
errores_porc = 100*((ensamble.predict(X_test)-y_test))/y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print ("Con % \nError maximo:{0}%\nError minimo:{1}%".format( max(abs(errores_porc)),min(abs(errores_porc))))
print(errores)

Error maximo:860407.6875
Error minimo:0.34375
Con % 
Error maximo:11875.3989583%
Error minimo:0.000366071428571%
7289        8144.492188
32437    -123436.250000
74570      32834.281250
62118       1515.750000
32002     -24489.671875
104773     15339.140625
7654       25830.957031
59082      11401.523438
120670     -1485.937500
26189      57991.625000
32701       -730.546875
45741        470.812500
10458     -20013.250000
51937       5507.320312
65439     -65251.140625
50809     -17905.078125
43894     -17119.656250
51308       -433.375000
52880      45434.875000
68709    -611625.187500
83652        573.156250
105896     17945.437500
8426       62533.867188
21807     -96971.093750
95500       -902.843750
80359      50310.812500
97124      40339.140625
74297      40456.656250
51500      -1558.843750
46497       7959.343750
              ...      
61393    -206822.312500
121150     51314.625000
121080       150.687500
25366      44032.750000
46127     -26456.585938
92024     -13550.042969

In [52]:
count_max=0
max_error=100
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 796 de 26557 que tienen mas de 100 % de error (2%)


In [53]:
count_max=0
max_error=20
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 5595 de 26557 que tienen mas de 20 % de error (21%)


In [54]:
count_max=0
max_error=10
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 7840 de 26557 que tienen mas de 10 % de error (29%)


In [55]:
count_max=0
max_error=5
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 9632 de 26557 que tienen mas de 5 % de error (36%)


### Errores arriba del maximo. No esta funcionando se cuelga todo

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(lista, 500, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-10000000, 10000000) #para variar el "zoom a 0"
plt.yscale('log')
plt.show()

### Errores totales

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(errores, 500, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-10000000, 10000000) #para variar el "zoom a 0"
plt.yscale('log')
plt.xscale('log')
plt.show()

# Kaggle

In [57]:
test_samples=pd.read_csv("datos/properati_dataset_testing_noprice_acondicionado.csv")
test_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 46 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        14166 non-null float64
surface_covered_in_m2      14166 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
cochera                    14166 non-null bool
comedor  

In [58]:
#paso features con strings a numericos

test_samples['place_name'] = le_barrio.transform(test_samples['place_name'])

test_samples['property_type'] = le_tipo.transform(test_samples['property_type'])

In [59]:
test= test_samples.loc[:,columnas]  

In [60]:
resultados = ensamble.predict(test)
result=pd.DataFrame({'id':test_samples['id'],'price_usd':resultados})
result.to_csv('datos/resultados.csv',index=False)